In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
results=pd.read_csv("C:\\Users\\Admin\\Desktop\\FakeNews\\results.csv")
test_data=pd.read_csv("C:\\Users\\Admin\\Desktop\\FakeNews\\test.csv")

In [ ]:
test_data.head()

In [ ]:
results.head()

## Checking For Empty values in the Test set

In [ ]:
print("Number of empty rows:{}".format(test_data.isna().sum().sum()))

In [ ]:
print("Number of empty rows in results:{}".format(results.isna().sum().sum()))

In [ ]:
test_data.fillna('NA',inplace=True)

In [ ]:
nan_filter=(test_data['title']=='NA')
nan_title_data=test_data.loc[nan_filter]
nan_title_ids=nan_title_data['id']
print(len(nan_title_ids))

In [ ]:
test_data.set_index('id',inplace=True)
results.set_index('id',inplace=True)

In [ ]:
test_data.drop(index=nan_title_ids,inplace=True)

In [ ]:
results.drop(index=nan_title_ids,inplace=True)

## Loading model file and vocabulary

In [ ]:
from keras import models 
news_predictor=models.load_model('news_predictor.h5') ## Loading Back the model file


In [ ]:
import pickle

with open('tokenizer.pkl','rb') as f:
    
    tokens=pickle.load(f)

### Predictions!!

In [ ]:
X_test=test_data['title']
X_id=test_data.index.values.tolist() 

In [ ]:
len(X_test)

In [ ]:
len(X_id)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer

In [ ]:
def preprocess_text(test_input):
    
    news_data=re.sub('[^a-zA-Z]',' ',test_input)
    news_data=news_data.lower()
    news_data=news_data.split()
    ps=PorterStemmer()
    news_data=[ps.stem(word) for word in news_data if word not in set(stopwords.words('english'))]
    news_data=' '.join(news_data)
    
    return news_data

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
 tokens.word_index

In [ ]:
def add_tokens(processed_news_data):
    
    vocab_size=15000
    
    processed_news_data=[processed_news_data]
    token_news_data=tokens.texts_to_sequences(processed_news_data)
        
    return token_news_data

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
def add_padding(encoded_news_data):
    
    sentence_length=47
    padded_news_data=pad_sequences(encoded_news_data,padding="post",maxlen=sentence_length)
    return padded_news_data

In [ ]:
import csv

In [ ]:
with open('news_results.csv','a',newline='') as f:
        
    csv_writer=csv.writer(f)
    csv_writer.writerow(['ID','Prediction'])

In [ ]:
X_test=list(X_test)

In [ ]:
for i in range(0,len(X_test)):
    
    test_input=X_test[i]
    input_id=X_id[i]
    processed_input=preprocess_text(test_input)
    token_input=add_tokens(processed_input)
    padded_news_input=add_padding(token_input)
    news_input=np.array(padded_news_input)
    news_result=news_predictor.predict(news_input)
    if news_result>0.5:
        
        result_value=1
        
    else:
        
        result_value=0
    with open('news_results.csv','a',newline='') as f:
        
       csv_writer=csv.writer(f)
       csv_writer.writerow([input_id,result_value])
        
        

# Classification Report

In [ ]:
prediction=pd.read_csv('news_results.csv')

In [ ]:
prediction.head()

In [ ]:
predicted_vals=prediction['Prediction']
predicted_vals=np.array(predicted_vals)

In [ ]:
predicted_vals

In [ ]:
len(predicted_vals)

In [ ]:
actual_vals=results['label']
actual_vals=np.array(actual_vals)

In [ ]:
actual_vals

In [ ]:
len(actual_vals)

In [ ]:
values_compare=np.concatenate((actual_vals.reshape(len(actual_vals),1),predicted_vals.reshape(len(predicted_vals),1)),1)
values_compare

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score
cm=confusion_matrix(actual_vals,predicted_vals)
print("Confusion Matrix: ")
print(" ")
print(cm)
print("")
acs=accuracy_score(actual_vals,predicted_vals)
acs=round(acs,2)*100
print("The accuracy score:{}%".format(acs))
print("")
precision=precision_score(actual_vals,predicted_vals)
precision=round(precision,2)*100
print("The precision score:{}%".format(precision))
print("")
recall=recall_score(actual_vals,predicted_vals)
recall=round(recall,2)*100
print("The recall score:{}%".format(recall))